Collecting data
===

# Step 1: find dates and locations of sandstorms

- Go to [NASA WorldView](https://worldview.earthdata.nasa.gov/?v=42.53496283204216,20.198112059324743,73.78467362462987,34.23809673633836&l=Reference_Labels_15m(hidden),Reference_Features_15m(hidden),Coastlines_15m(hidden),VIIRS_SNPP_CorrectedReflectance_TrueColor&lg=true&t=2022-01-21-T13%3A06%3A55Z)
- Paste the below JavaScript into the dev console
- Click on some sandstorms

```js
/**
 * This JS code modifies the NASA WORLDVIEW webpage
 * (https://worldview.earthdata.nasa.gov/) to enable logging of timestamps and
 * coordinates when any point on the map is clicked.
 * 
 * This is achieved through event listeners. Additionally, clicks on the
 * sidebar and timeline, as well as long presses and drags, are ignored.
 * 
 * To use this code, paste it into the web browser's dev console and hit
 * enter.
 */

const timeline_elem = document.querySelector('div.timeline-container');
const sidebar_elem = document.getElementById('wv-sidebar');
var is_drag = false;
var is_long_press = false;
var timeoutId;
function handleMouseDown() {
	is_drag = false;
	timeoutId = setTimeout(function () { is_long_press = true; }, 250);
}
function handleMouseMove() {
	is_drag = true;
}
function log_date_coords(event) {
	clearTimeout(timeoutId);
	const drag_or_long_press = is_drag || is_long_press;
	is_drag = false;
	is_long_press = false;
	if (drag_or_long_press) return;
	if (timeline_elem.contains(event.target)) return;
	if (sidebar_elem.contains(event.target)) return;
	const coords = document.querySelector('span.map-coord').innerText;
	const coords_list = coords.split(',');
	const lat = coords_list[0].trim().substring(0, coords_list[0].length - 1);
	const lon = coords_list[1].trim().substring(0, coords_list[0].length - 1);
	const year = document.getElementById('year-timeline').value;
	const month = document.getElementById('month-timeline').value;
	const day = document.getElementById('day-timeline').value;
	const date = `${year}-${month}-${day}`;
	const msg = `${date},${lat},${lon}`;
	console.log(msg);
};
document.getElementById('wv-content').addEventListener('mouseup', log_date_coords);
document.getElementById('wv-content').addEventListener('mousedown', handleMouseDown);
document.getElementById('wv-content').addEventListener('mousemove', handleMouseMove);
```

# Step 2: download imagery

In [ ]:
from skimage import io
from rastervision.core.box import Box


def get_img(layer_name: str, date: str | None, bbox: Box, height: int,
            width: int, crs: str) -> 'np.ndarray':
    bbox_str = ','.join(map(str, bbox))
    args = dict(
        version='1.3.0',
        service='WMS',
        request='GetMap',
        format='image/png',
        STYLE='default',
        bbox=bbox_str,
        CRS=crs,
        HEIGHT=height,
        WIDTH=width,
        layers=layer_name,
    )
    if date is not None:
        args['TIME'] = date
    query_string = '&'.join(f'{k}={v}' for k, v in args.items())
    url = f'https://gibs.earthdata.nasa.gov/wms/epsg4326/best/wms.cgi?{query_string}'
    arr = io.imread(url)
    return arr

In [ ]:
layer_name = 'VIIRS_SNPP_CorrectedReflectance_TrueColor'
bbox = Box(ymin=24, xmin=54, ymax=34, xmax=64)
date = '2022-01-21'

arr = get_img(
    layer_name=layer_name,
    date=date,
    bbox=bbox,
    height=1200,
    width=1200,
    crs='EPSG:4326',
)

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(arr)
plt.show()

In [ ]:
from os.path import join
from rastervision.pipeline.file_system.utils import make_dir
from rastervision.core.data.utils import write_bbox

bbox_str = ','.join(map(str, bbox))
tiff_path = join('data/gibs/img/', f'{date}_{bbox_str}.tif')
make_dir(tiff_path, use_dirname=True)
write_bbox(tiff_path, arr, bbox, crs_wkt='epsg:4326')

# Step 3: label data

> Become one with the data

― Andrej Karpathy, [_A Recipe for Training Neural Networks_](https://karpathy.github.io/2019/04/25/recipe/)

`<demo>`